In [12]:
from keras.models import Model, Sequential
from keras.layers import Flatten, Lambda
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.optimizers import Adam

import numpy as np

# VGG Setup

In [2]:

vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((1,1,3))
def vgg_preprocess(x):
    """
        Subtracts the mean RGB value, and transposes RGB to BGR.
        The mean RGB was computed on the image set used to train the VGG model.

        Args: 
            x: Image array (height x width x channels)
        Returns:
            Image array (height x width x transposed_channels)
    """
#     x = x - vgg_mean
#     return x[:, ::-1] # reverse axis rgb->bgr
    return x

# img_input = Input(shape=(224, 224, 3))

model = Sequential()
model.add(Lambda(vgg_preprocess, input_shape=(224,224,3), output_shape=(224,224,3)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))
model.add(Flatten(name='flatten'))
model.add(Dense(4096, activation='relu', name='fc1'))
model.add(Dense(4096, activation='relu', name='fc2'))
model.add(Dense(1000, activation='softmax', name='predictions'))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [3]:
from keras.utils.data_utils import get_file

file_name = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'
# file_name = 'vgg16.h5'
file_path = 'http://files.fast.ai/models/'
model.load_weights(get_file(file_name, file_path+file_name, cache_subdir='models'))

In [4]:
for layer in model.layers:
    layer.trainable = False

model.pop()
model.add(Dense(2, activation='softmax', name='predictions'))

model.summary()

model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

# ResNet 50 Setup

In [2]:
from keras.applications.resnet50 import ResNet50

In [3]:
resnet = ResNet50(weights='imagenet', include_top=False)
resnet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, None, None, 3) 0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, None, None, 64 256         conv1[0][0]                      
___________________________________________________________________________________________

In [4]:
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import Adam, RMSprop

resnet.layers.pop()
for layer in resnet.layers: layer.trainable=False
m = GlobalAveragePooling2D()(resnet.layers[-1].output)
m = Dropout(0.5)(m)
m = Dense(2, activation='softmax')(m)
my_model = Model(resnet.input, m)
my_model.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

my_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, None, None, 3) 0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, None, None, 64 256         conv1[0][0]                      
___________________________________________________________________________________________

# Training

In [5]:
#Set constants. You can experiment with no_of_epochs to improve the model
#Set path to sample/ path if desired
test_path = 'testing/' #We use all the test data
train_path='train/'
valid_path='valid/'

batch_size=32
no_of_epochs=1

In [11]:
model.load_weights('ensemble_vgg16.h5')
my_model.load_weights('ensemble_resnet.h5')


In [6]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_data = datagen.flow_from_directory(train_path, target_size=(224,224),
                class_mode='categorical', shuffle=True, batch_size=batch_size)

valid_data = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224),
                class_mode='categorical', shuffle=True, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
import keras
from keras import backend as K
# K.clear_session()
    
for epoch in range(no_of_epochs):
    steps_per_epoch = int(train_data.samples/train_data.batch_size)

    if steps_per_epoch == 0:
        steps_per_epoch = 1

    validation_steps = int(valid_data.samples/valid_data.batch_size)

    if validation_steps == 0:
        validation_steps = 1
        
#     model.fit_generator(train_data, 
#                              steps_per_epoch=steps_per_epoch,
#                              epochs=1,
#                              validation_data=valid_data, 
#                              validation_steps=validation_steps)
    
#     model.save_weights('ensemble1.h5')     

    my_model.fit_generator(train_data, 
                             steps_per_epoch=steps_per_epoch,
                             epochs=1,
                             validation_data=valid_data, 
                             validation_steps=validation_steps)
    
#     my_model.save_weights('ensemble1.h5')         

print("Completed", no_of_epochs, "fit operations")

Epoch 1/1
718/718 [==============================] - 320s - loss: 1.8789 - acc: 0.8750 - val_loss: 0.5881 - val_acc: 0.9612
Completed 1 fit operations


In [8]:
my_model.save_weights('ensemble_resnet2.h5')  

In [12]:
from keras.preprocessing.image import ImageDataGenerator

test_path = "testing/"

gen = ImageDataGenerator()
images = gen.flow_from_directory(test_path, target_size=(224,224), class_mode='categorical', shuffle = False)
predict1 = model.predict_generator(images, images.samples // images.batch_size + 1)


Found 12500 images belonging to 1 classes.


In [10]:
gen = ImageDataGenerator()

images = gen.flow_from_directory(test_path, target_size=(224,224), class_mode='categorical', shuffle = False)
predict2 = my_model.predict_generator(images, images.samples // images.batch_size + 1)


Found 12500 images belonging to 1 classes.


In [26]:
predict = (predict1 + predict2) / 2

In [14]:
import pandas as pd

submission = pd.DataFrame()
submission['label'] = predict2[0:, 1]

test_list = []

for fn in range(len(images.filenames)):
    test_list.append(images.filenames[fn].replace('test1\\', '').replace('.jpg', ''))
    
submission['id'] = pd.to_numeric(test_list)

submission.sort_values(["id"], inplace = True)

submission['label'][submission['label'] < 0.05] = 0.05
submission['label'][submission['label'] > 0.95] = 0.95

submission.to_csv('submission.csv', columns = ["id", "label"], index = False)


C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [ ]:
sub1 = pd.read_csv('submission.csv')
sub2 = pd.read_csv('submission2.csv')

final_sub = (sub1 + sub2) / 2

sub1['label'] = final_sub['label']
sub1.to_csv('final.csv', columns = ["id", "label"], index = False)